In [ ]:
# Import modules
import cv2
import numpy as np
import os
#from google.colab import drive

# Mount google drive
#drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Set categories
categories = ["Faces", "cougar_face", "flamingo", "nautilus", "sunflower", "Faces_easy", "dalmatian", "headphone", "pagoda", "tick", "brontosaurus", "dollar_bill", "lotus", "soccer_ball", "windsor_chair", "car_side", "dragonfly", "menorah", "stop_sign", "yin_yang"]

# Configure path
#base_path = "/content/gdrive/My Drive/Colab Notebooks"
base_path = os.getcwd()
data_path = base_path + "/dataset"

In [ ]:
# Initialize detector for feature extraction
detector = cv2.ORB_create()

In [ ]:
# Set empty arrays
train_path = []
train_label = []
train_features = np.array([])

# Set # of images in one category
image_length = 30
count = 0

In [ ]:
# Extract features from data images
for index, item in enumerate(categories):
  
  category_dir = data_path + "/train/" + item

  for index2 in range(image_length):

    image_path = category_dir + "/image_%04d.jpg" % (index2+1)
    train_path.append(image_path)
    train_label.append(index)
    img = cv2.imread(image_path)

    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    kpt, desc = detector.detectAndCompute(gimg, None)

    if train_features.size == 0:
      train_features = np.float32(desc)
    else:
      train_features = np.append(train_features, np.float32(desc), axis=0)

    count += 1
    print('%d/%d - %s - %d feature points are detected\n' % (count, image_length*20, image_path, desc.shape[0]))

1/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0001.jpg - 492 feature points are detected

2/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0002.jpg - 500 feature points are detected

3/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0003.jpg - 495 feature points are detected

4/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0004.jpg - 500 feature points are detected

5/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0005.jpg - 500 feature points are detected

6/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0006.jpg - 500 feature points are detected

7/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0007.jpg - 500 feature points are detected

8/600 - /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0008.jpg - 498 feature points are detected

9/600 - /content/gdrive/My Drive/Colab Notebooks

In [ ]:
# Generate dictionary by using K-means clustering
dictionary_size = 10
dict_path = base_path + "/4dictionary.npy"

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)
ret, label, dictionary = cv2.kmeans(train_features, dictionary_size, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

np.save(dict_path, dictionary)

In [ ]:
# Generate K-nearest model and train dictionary
knn = cv2.ml.KNearest_create()
knn.train(dictionary, cv2.ml.ROW_SAMPLE, np.float32(range(dictionary_size)))
train_desc = np.float32(np.zeros((len(train_path), dictionary_size)))

In [ ]:
# Generate histogram by applying K-nearest model.
for index, path in enumerate(train_path):

  img = cv2.imread(path)
  gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  kpt, desc = detector.detectAndCompute(gimg, None)
  ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)

  hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))
  # Normalize histogram data
  train_desc[index, :] = np.float32(hist) / np.float32(np.sum(hist))

  print('%d/%d - Representing %s \n' % (index+1, len(train_path), path))

1/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0001.jpg 

2/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0002.jpg 

3/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0003.jpg 

4/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0004.jpg 

5/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0005.jpg 

6/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0006.jpg 

7/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0007.jpg 

8/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0008.jpg 

9/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0009.jpg 

10/600 - Representing /content/gdrive/My Drive/Colab Notebooks/dataset/train/Faces/image_0010.jpg 

11/600 - 

In [ ]:
# Apply K-Nearest instead of SVM
kn_model_file = base_path + "/nearest_neighbor.xml"
kn = cv2.ml.KNearest_create()
kn.train(train_desc, cv2.ml.ROW_SAMPLE, np.array(train_label))
kn.save(kn_model_file)

In [ ]:
# Set variables and path for test data
test_path = data_path + "/test"
test_length = 5
total_test = test_length * len(categories)
correct = 0

In [ ]:
# Test BOW model with KNN (instead of SVM)
for index, item in enumerate(categories):

  target_dir = test_path + "/" + item

  for index2 in range(test_length):

    test_desc = np.float32(np.zeros((1, dictionary_size)))
    image_path = target_dir + "/image_%04d.jpg" % (31 + index2)

    # Generate histogram from test data
    img = cv2.imread(image_path)
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kpt, desc = detector.detectAndCompute(gimg, None)
    ret, result, nbs, dist = knn.findNearest(np.float32(desc), k=1)
    hist, bins = np.histogram(np.int32(result), bins=range(dictionary_size + 1))

    # Apply normalized histogram in K-Nearest
    test_desc[0, :] = np.float32(hist) / np.float32(np.sum(hist))
    ret, result = kn.predict(test_desc)

    if item == categories[int(result[0][0])]:
      correct += 1

In [ ]:
# Calculate accuracy
accuracy = correct / total_test
print(accuracy)

0.24
